In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import time
import matplotlib.pyplot as plt

from keras.callbacks import History 
history = History()

BATCH_SIZE = 10

number_of_attributes = 11

print(tf.__version__)

2.7.0


In [3]:
path_training ="/content/drive/MyDrive/IC-Colab-CSV files/01_Atributos_Weve.csv"
path_test ="/content/drive/MyDrive/IC-Colab-CSV files/02_Atributos_Weve.csv"
df = pd.read_csv(path_training)
df_test = pd.read_csv(path_test)
df
target = df.pop('target')
server_port = df.pop('ServerPort')

target_test = df_test.pop('target')
server_port_test = df_test.pop('ServerPort')


In [4]:
#df['15'].hist()
#df.dtypes

In [5]:
#array_de_atributos = ['60', '59', '82', '46', '17', '84', '79', '10', '90', '15', '24', '92', '91', '83', 'ClientPort', '48', '37', '22', '88', '47', '45', '34', '32', '36', '38', '33', '31', '80', '61', '62', '64', '89', '63', '29', '87', '81', '75','76','77','78']
array_de_atributos = ['60', '59', '82', '46', '17', '84', '79', '90', '15', '24', '83', '48', '37', '22', '88', '47', '45', '34', '32', '36', '38', '33', '31', '80', '61', '62', '64', '89', '63', '29', '87', '81','92','91', '10', 'ClientPort' '75','76','77','78']                     

In [6]:
df = df[array_de_atributos[0:number_of_attributes]]
df_test = df_test[array_de_atributos[0:number_of_attributes]]
df

,60,59,82,46,17,84,79,90,15,24,83
0,1,1,140,1,40,140,1380,8192,373,14,319
1,1,1,141,1,40,141,1380,8192,373,14,319
2,1,1,141,1,40,141,1380,8192,376,14,322
3,1,1,142,1,40,142,1380,8192,373,14,319
4,1,1,141,1,40,141,1380,8192,379,14,325
...,...,...,...,...,...,...,...,...,...,...,...
24858,4,3,85,4,40,8,1460,49640,139,14,20
24859,7,7,73,7,52,21,1460,57920,1061,14,6
24860,7,5,97,7,52,8,1460,49232,163,14,6
24861,1,1,536,45,40,330,1460,8192,590,14,119


In [7]:
classes = ['WWW','MAIL','FTP-CONTROL','FTP-PASV','ATTACK','P2P','DATABASE','FTP-DATA','MULTIMEDIA','SERVICES','INTERACTIVE','GAMES']
#classes = list(target.unique())
#classes
#Converts the above list of classes to an array of index
target = target.apply(lambda x: classes.index(x))
target.unique()
target_test = target_test.apply(lambda x: classes.index(x))

In [8]:
#como automatizar o numeric_feature_names a partir de um parâmetro passado com os atributos escolhidos?
#['ClientPort','10','15','17','22','24','29','31','32','33','34','36','37','38','45','46','47','48','59','60','61','62','63','64','75','76','77','78','79','80','81','82','83','84','87','88','89','90','91','92']

numeric_feature_names = array_de_atributos[0:number_of_attributes]
numeric_features = df[numeric_feature_names] # Data for training
numeric_features_test = df_test[numeric_feature_names] # Data for testing
numeric_features.head()

,60,59,82,46,17,84,79,90,15,24,83
0,1,1,140,1,40,140,1380,8192,373,14,319
1,1,1,141,1,40,141,1380,8192,373,14,319
2,1,1,141,1,40,141,1380,8192,376,14,322
3,1,1,142,1,40,142,1380,8192,373,14,319
4,1,1,141,1,40,141,1380,8192,379,14,325


In [9]:
#converting pandas dataframe to tensorflow object
numeric_features = tf.convert_to_tensor(numeric_features)
numeric_features_test = tf.convert_to_tensor(numeric_features_test)
numeric_features

<tf.Tensor: shape=(24863, 11), dtype=int64, numpy=
array([[  1,   1, 140, ..., 373,  14, 319],
       [  1,   1, 141, ..., 373,  14, 319],
       [  1,   1, 141, ..., 376,  14, 322],
       ...,
       [  7,   5,  97, ..., 163,  14,   6],
       [  1,   1, 536, ..., 590,  14, 119],
       [  4,   3,  76, ..., 130,  14,  20]])>

In [10]:
# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)
#normalizer.adapt(numeric_features_test)

Ideia: Fazer a normalização entre 0 e 1 conforme o Andrews, aqui atualmente ele está centrado em 0 com desvio padrão em 1

In [11]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [12]:
model = get_basic_model()
history = model.fit(numeric_features, target, epochs=10, batch_size=BATCH_SIZE, callbacks=[history])



Epoch 1/10
2487/2487 [==============================] - 5s 2ms/step - loss: 0.5031 - accuracy: 0.8886
Epoch 2/10
2487/2487 [==============================] - 5s 2ms/step - loss: 0.2053 - accuracy: 0.9449
Epoch 3/10
2487/2487 [==============================] - 5s 2ms/step - loss: 0.1633 - accuracy: 0.9536
Epoch 4/10
2487/2487 [==============================] - 5s 2ms/step - loss: 0.1432 - accuracy: 0.9602
Epoch 5/10
2487/2487 [==============================] - 5s 2ms/step - loss: 0.1311 - accuracy: 0.9640
Epoch 6/10
2487/2487 [==============================] - 5s 2ms/step - loss: 0.1238 - accuracy: 0.9673
Epoch 7/10
2487/2487 [==============================] - 4s 2ms/step - loss: 0.1201 - accuracy: 0.9692
Epoch 8/10
2487/2487 [==============================] - 4s 2ms/step - loss: 0.1155 - accuracy: 0.9701
Epoch 9/10
2487/2487 [==============================] - 4s 2ms/step - loss: 0.1124 - accuracy: 0.9708
Epoch 10/10
2487/2487 [==============================] - 4s 2ms/step - loss: 0.109

In [13]:
#print(history.history)
#print("\n")
train_accuracy = history.history['accuracy']
#print(train_accuracy)

# xc = range(10)
# plt.figure()
# plt.plot(xc, train_accuracy)

In [14]:
test_loss, test_acc = model.evaluate(numeric_features_test,  target_test, verbose=2, batch_size=BATCH_SIZE) 
print('\nTest accuracy:', test_acc)


2381/2381 - 3s - loss: 0.1876 - accuracy: 0.9564 - 3s/epoch - 1ms/step

Test accuracy: 0.9564303755760193


In [15]:
#layer0_weights = model.layers[0].get_weights()[0] #weight
#layer0_biases  = model.layers[0].get_weights()[1] #bias
layer1_weights = model.layers[1].get_weights()[0] #weight
#layer1_biases  = model.layers[1].get_weights()[1] #bias
print('Layer 1 weights:',layer1_weights)
# print('\n')

weight_attribute = []
for attr_i, weights in enumerate(layer1_weights):
  weight_attribute.append((weights.sum(), numeric_feature_names[attr_i]))

weight_attribute = sorted(weight_attribute)

print(weight_attribute)

attributes = [x for _, x in weight_attribute]

print(attributes)

Layer 1 weights: [[ 0.67523724  1.0221695  -0.5484712  -0.897517   -0.08093134  0.21693522
  -1.999157    0.8014047  -0.4756433  -4.401109  ]
 [-0.6777456  -1.5421666   0.7178528  -0.37459388  0.22910039  0.05140788
  -0.9459743   0.20134418  0.5490977  -1.413662  ]
 [ 0.8355146  -0.86015326 -1.50016     0.15120181 -0.7243978   0.9466476
  -0.24657129  0.17266971  0.23411804 -1.5722839 ]
 [-0.25186747 -0.4157875  -1.599906   -0.19630153  0.25541458 -0.00509225
  -0.298887   -0.4826092   0.28581214 -1.1264274 ]
 [-1.2939974   0.34388238  0.127684   -1.6508437   0.5838523   0.5761736
   0.5386551   0.561624    0.9938568  -0.20264174]
 [ 0.21325953 -0.2470908  -1.1653252   0.00926308  0.3974941   0.3559463
   0.62852025 -0.526509   -0.07239004 -0.7736529 ]
 [ 0.3392223  -0.37845245 -0.5033974   0.9792534  -0.7531562   0.8627794
  -0.19872226  0.20221598 -0.01284408 -0.06134611]
 [-0.6007503   0.0988439  -0.37888706  0.40211326 -0.17947894  1.1237949
  -0.5343591  -0.15328059 -2.2946098   

In [16]:
#print('\nPesos da primeira camada')
#print(model.layers[0].weights)
#print('\nPesos da segunda camada: intermediária')
##print(model.layers[1].weights)
#print('\nPesos da terceira camada: final')
#print(model.layers[2].weights)

In [17]:
current_date_and_time = datetime.date.today()
current_date_and_time_string = str(current_date_and_time)

report = {
    "datetime": current_date_and_time_string,
    "training_dataset_shape":  df.shape,
    "test_dataset_shape": df_test.shape,
    "attributes": array_de_atributos[0:df.shape[1]],
    "number_of_attributes": number_of_attributes,
    "accuracy_train": train_accuracy[-1],
    "accuracy_test": test_acc
}

titulo = "/content/drive/MyDrive/IC-Skynet/NNreports/%s"%current_date_and_time_string + " With %d Attributes2.json"%number_of_attributes
titulo

with open(titulo, "w") as f:
  json.dump(report, f)

with open(titulo, "r") as f:
  rep = json.load(f)

display(rep)

#print(rep["datetime"])




{'accuracy_test': 0.9564303755760193,
 'accuracy_train': 0.9708402156829834,
 'attributes': ['60',
  '59',
  '82',
  '46',
  '17',
  '84',
  '79',
  '90',
  '15',
  '24',
  '83'],
 'datetime': '2022-01-10',
 'number_of_attributes': 11,
 'test_dataset_shape': [23801, 11],
 'training_dataset_shape': [24863, 11]}